In [ ]:
import numpy as np
import pandas as pd

c_2015 = pd.read_csv('./data/Comparendos/Comparendos_DEI_2015_BogotA1_D.C.csv', encoding = 'utf8', low_memory=False)
c_2016 = pd.read_csv('./data/Comparendos/Comparendos_DEI_2016_BogotA1_D.C.csv', encoding = 'utf8', low_memory=False)
c_2017 = pd.read_csv('./data/Comparendos/Comparendos_DEI_2017_BogotA1_D.C.csv', encoding = 'utf8', low_memory=False)
c_2018 = pd.read_csv('./data/Comparendos/Comparendos_DEI_2018_BogotA1_D.C.csv', encoding = 'utf8', low_memory=False)
c_2019 = pd.read_csv('./data/Comparendos/Comparendos_2019_Bogota_D_C.csv', encoding = 'utf8', low_memory=False)
c_2020 = pd.read_csv('./data/Comparendos/Comparendos_DEI_2020_BogotA1_D_C.csv', encoding = 'utf8', low_memory=False)
c_2021 = pd.read_csv('./data/Comparendos/Comparendos_2021_Bogota_D_C.csv', encoding = 'utf8', low_memory=False)

In [ ]:
columns=["OBJECTID","HORA_OCURRENCIA","MUNICIPIO"]

c_2015.drop(columns, axis=1, inplace=True)
c_2015.rename(columns={"Y":"LATITUD", "X":"LONGITUD"}, inplace=True, errors="raise")

In [ ]:
columns=["OBJECTID"]

c_2016.drop(columns, axis=1, inplace=True)
c_2016.rename(columns={"Y":"LATITUD", "X":"LONGITUD"}, inplace=True, errors="raise")

In [ ]:
columns=["OBJECTID"]

c_2017.drop(columns, axis=1, inplace=True)
c_2017.rename(columns={"Y":"LATITUD", "X":"LONGITUD"}, inplace=True, errors="raise")

In [ ]:
columns=["OBJECTID","MUNICIPIO"]

c_2018.drop(columns, axis=1, inplace=True)
c_2018.rename(columns={"Y":"LATITUD", "X":"LONGITUD"}, inplace=True, errors="raise")

In [ ]:
columns=["X", "Y","OBJECTID","NUM_COMPARENDO", "ANO", "HORA_OCURRENCIA", "MES", "MUNICIPIO", "GlobalID"]

c_2019.drop(columns, axis=1, inplace=True)
c_2019.head()

In [ ]:
columns=["X", "Y", "FID", "OBJECTID", "ANO", "HORA_OCURR", "MES", "MUNICIPIO", "PAIS"]

c_2020.drop(columns, axis=1, inplace=True)
c_2020.rename(columns={"MEDIO_DETE":"MEDIO_DETECCION", "CLASE_VEHI":"CLASE_VEHICULO", "TIPO_SERVI":"TIPO_SERVICIO", "DES_INFRAC":"DES_INFRACCION"},
              inplace=True, errors="raise")

In [ ]:
columns=["X", "Y", "FID", "OBJECTID","NUM_COMPAR", "ANO", "HORA_OCURR", "MES", "DIRECCION", "MUNICIPIO"]

c_2021.drop(columns, axis=1, inplace=True)
c_2021.rename(columns={"MEDIO_DETE":"MEDIO_DETECCION", "CLASE_VEHI":"CLASE_VEHICULO", "TIPO_SERVI":"TIPO_SERVICIO", "DES_INFRAC":"DES_INFRACCION"},
              inplace=True, errors="raise")

In [ ]:
comparendos = pd.concat([c_2015,c_2016,c_2017,c_2018,c_2019,c_2020,c_2021], ignore_index=True)

dia_semana={0:'LUNES', 1:'MARTES', 2:'MIERCOLES', 3:'JUEVES', 4:'VIERNES', 5:'SABADO', 6:'DOMINGO'}
mes={1:'ENERO', 2:'FEBRERO', 3:'MARZO', 4:'ABRIL', 5:'MAYO', 6:'JUNIO',
     7:'JULIO', 8:'AGOSTO', 9:'SEPTIEMBRE', 10:'OCTUBRE', 11:'NOVIEMBRE', 12:'DICIEMBRE'}

comparendos['FECHA_HORA']=pd.to_datetime(comparendos['FECHA_HORA'])
comparendos['HORA'] = comparendos['FECHA_HORA'].dt.hour
comparendos['DIA'] = comparendos['FECHA_HORA'].dt.day
comparendos['DIA_SEMANA'] = comparendos['FECHA_HORA'].dt.dayofweek.map(dia_semana)
comparendos['MES'] = comparendos['FECHA_HORA'].dt.month.map(mes)
comparendos['ANO'] = comparendos['FECHA_HORA'].dt.year
comparendos['LOCALIDAD'] = comparendos['LOCALIDAD'].str.replace(r'\d+-', '')
comparendos['LOCALIDAD'] = comparendos['LOCALIDAD'].str.strip()

comparendos.replace({'ANTONIO NARIÑO': 'ANTONIO NARINO', 'Field was not enabled':'', '-':'','LA CANDELARIA':'CANDELARIA',
                     'MARTIRES':'LOS MARTIRES', 'RAFAEL URIBE':'RAFAEL URIBE URIBE'}, inplace=True)
comparendos.replace({'Diplomático':'Diplomatico'}, inplace=True)

In [ ]:
import pygeohash as gh

comparendos['CLASE_VEHICULO'] = comparendos['CLASE_VEHICULO'].str.strip()
comparendos.replace({'AUTOMOVIL': 'AUTOMÓVIL', 'CAMION':'CAMIÓN', 'CICLOMOTO':'CICLOMOTOR','MOTOTRICILO':'MOTOTRICICLO',
                     'TRACCION ANIMAL':'TRACCIÓN ANIMAL', 'TRACTOCAMION':'TRACTOCAMIÓN', '-1':''}, inplace=True)

comparendos['CLASE_VEHICULO'] = comparendos['CLASE_VEHICULO'].str.upper()
comparendos["GEO"] = comparendos['LATITUD'].astype(str)+','+comparendos['LONGITUD'].astype(str)
comparendos["GEOHASH"] = comparendos.apply(lambda x: gh.encode(x.LATITUD, x.LONGITUD, precision=6), axis=1)

In [ ]:
comparendos.replace({'': None, ' ': None}, inplace=True) 
comparendos.replace({np.nan: None}, inplace=True)
comparendos.fillna({0: None}, inplace=True)

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

NODES = ["https://myelasticinstance01:9200","https://myelasticinstance02:9200"]
es_client = Elasticsearch(
    NODES,
    #verify_certs=False,
    #ssl_show_warn=False,
    basic_auth=("myawesomename", "mysupersecretpassword")
)

es_client.info()

In [ ]:
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        doc = {
            "_index" : "movilidad_comparendos",
            "_id" : index,
            "_source" : document.to_dict()
        }
        yield doc
        

In [ ]:
for success, info in helpers.parallel_bulk(es_client, doc_generator(comparendos)):
    if not success:
        print('A document failed:', info)